<a href="https://akademie.datamics.com/kursliste/">![title](bg_datamics_top.png)</a>

<center><em>© Datamics</em></center><br><center><em>Besuche uns für mehr Informationen auf <a href='https://akademie.datamics.com/kursliste/'>www.akademie.datamics.com</a></em>

# Projekt Capstone

## Finger erkennen und zählen

## Importe

In [1]:
import cv2
import numpy as np

# Wird für die Distanzberechnung später gebraucht
from sklearn.metrics import pairwise

### Globale Variablen

# Wir werden diese später verwenden.

In [1]:
# Dieser Hintergrund wird eine globale Variable, die wir später durch ein paar Funktionen aktualisieren
background = None

# Starte mit dem Mittelwert zwischen 0 und 1 des akkumulierten Gewichtes
accumulated_weight = 0.5


# Setze die ROI manuell zum Finden der Hand.
# Ändere diese nach belieben. Ich habe nur die rechte obere Ecke zum Filmen gewählt.
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600



## Finde den durchschnittlichen Hintergrundwert

Diese Funktion berechnet die gewichtete Summe der Eingabebildquelle src und dem Akkumulator dst, so dass dst der laufende Mittelwert eine Framesequenz wird.

In [3]:
def calc_accum_avg(frame, accumulated_weight):
    '''
    Berechne den gemittelten Durchschnitt des übergebenen Bildes basierend auf dem vorher akkumuliertem Gewicht.
    '''
    
    # Nimm den Hintergrund
    global background
    
    # Erzeuge den Hintergrund aus einer Kopie des Frames beim ersten Mal
    if background is None:
        background = frame.copy().astype("float")
        return None

    # Berechne den gewichteten Mittelwert, akkumuliere ihn und aktualisiere den Hintergrund
    cv2.accumulateWeighted(frame, background, accumulated_weight)

## Segmentiere die Handregion im Frame

In [4]:
def segment(frame, threshold=25):
    global background
    
    # Berechne die absolute Differenz zwischen Hintergrund und dem übergebenen Frame
    diff = cv2.absdiff(background.astype("uint8"), frame)

    # Wende einen Grenzwert auf das Bild an, um den Vordergrund zu erhalten
    # Wir brauchen nur den Grenzwert, daher werfen wir das erste Element im Tupel mit dem Unterstrich _ weg
    _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    # Nimm die externen Konturen des Bildes
    # Auch hier wird nur genommen, was wir brauchen, und der Rest weggeworfen
    image, contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Wenn die Länge der Liste 0 ist, wurden keine Konturen gefunden!
    if len(contours) == 0:
        return None
    else:
        # So wie wir das Programm benutzen, sollte die größte, externe Kontur die Hand sein (nach Fläche)
        # Dies wird unser Segment sein
        hand_segment = max(contours, key=cv2.contourArea)
        
        # Gib sowohl das Handsegment als auch das Grenzwertbild der Hand zurück
        return (thresholded, hand_segment)

## Finger mit einer konvexen Hülle zählen

Wir haben gerade die externe Kontur einer Hand berechnet. Unter Verwendung dieser segmentierten Hand, lass uns herausfinden, wie die Finger gezählt werden können. Dann zählen wir, wie viele ausgestreckt sind!

Beispiele konvexer Hüllen:

<img src="hand_convex.png">

In [5]:
def count_fingers(thresholded, hand_segment):
    
    # Berechne die konvexe Hülle des Handsegmentes
    conv_hull = cv2.convexHull(hand_segment)
    
    # Die konvexe Hülle hat jetzt mindestens 4 äußerste Punkte, oben, unten, links und rechts.
    # Lass uns diese Punkte nehmen mit argmin und argmax. Bedenke, dies wird das Lesen der Dokumentation 
    # und das Verstehen der von convexHull zurückgegebenen allgemeinen Array-Form bedürfen.

    # Finde oben, unten, links und rechts
    # Stelle sicher, dass sie im Tupel-Format vorliegen
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])

    # In der Theorie ist der Handmittelpunkt mittig zwischen oben und unten und mittig zwischen links und rechts
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2

    # Finde die maximale euklidische Distanz zwischen dem Handmittelpunkt und den extremsten Punkten der konvexen Hülle

    # Berechne die euklidische Distanz zwischen dem Mittelpunkt der Hand und links, rechts, oben und unten.
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    
    # Nimm die größte Entfernung
    max_distance = distance.max()
    
    # Erzeuge einen Kreis mit 90% Radius der maximalen euklidischen Distanz
    radius = int(0.8 * max_distance)
    circumference = (2 * np.pi * radius)

    # Erzeuge jetzt einen ROI nur dieses Kreises
    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
    
    # Zeichne die kreisförmige ROI
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)
    
    
    # Verwende bit-weises AND mit dem ROI-Kreis als Maske
    # Dies gibt dann den Ausschnitt zurück von der Anwendung der Maske auf das Grenzwertbild.
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

    # Nimm die Konturen im ROI-Kreis
    image, contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Fingerzähler startet mit 0
    count = 0

    # Iteriere über die Konturen, um zu sehen, ob wir mehr Finger zählen können
    for cnt in contours:
        
        # Begrenzungsbox der Kontur
        (x, y, w, h) = cv2.boundingRect(cnt)

        # Erhöhe die Anzahl von Fingern basierend auf zwei Konditionen:
        
        # 1. Die Kontur ist nicht im untersten Bereich der Hand (Handgelenk)
        out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
        
        # 2. Die Anzahl von Punkten entlang der Kontur ist nicht größer als 25% des Umfangs des ROI-Kreises (sonst zählen wir Punkte von der Hand)
        limit_points = ((circumference * 0.25) > cnt.shape[0])
        
        
        if  out_of_wrist and limit_points:
            count += 1

    return count

## Programm ausführen

In [10]:
cam = cv2.VideoCapture(0)

# Intialisiere einen Framezähler
num_frames = 0

# Schleife läuft endlos bis zu einer Unterbrechung
while True:
    # Nimm den derzeitige Frame
    ret, frame = cam.read()

    # Drehe den Frame, so dass es nicht gespiegelt ist.
    frame = cv2.flip(frame, 1)

    # Kopiere den Frame
    frame_copy = frame.copy()

    # Nimm die ROI des Frames
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]

    # Konvertiere die ROI in Graustufen und füge Unschärfe hinzu
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # Für die ersten 30 Frames werden wir den Durchschnitt des Hintergrunds berechnen
    # Wir sagen das dem Nutzer, so lange es passiert
    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVG.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Finger Count",frame_copy)
            
    else:
        # Mit dem erhaltenen Hintergrund können wir jetzt die Hand segmentieren.
        
        # Segmentiere die Handregion
        hand = segment(gray)

        # Überprüfe zuerst, ob wir eine Hand erkannt haben
        if hand is not None:
            
            # Teile auf
            thresholded, hand_segment = hand

            # Zeichne Konturen um das Handsegment
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0),1)

            # Zähle die Finger
            fingers = count_fingers(thresholded, hand_segment)

            # Zeige die Anzahl an
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            # Zeige auch das Grenzwertbild an
            cv2.imshow("Thesholded", thresholded)

    # Zeichne das Rechteck des ROIs auf die Kopie des Frames
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)

    # Erhöhe den Framezähler
    num_frames += 1

    # Zeige das Frame mit segmentierter Hand
    cv2.imshow("Finger Count", frame_copy)


    # Schließe das Fenster mit Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

# Gib die Kamera frei und zerstöre alle Fenster
cam.release()
cv2.destroyAllWindows()